# A2 - OpenJij core interface入門 (C++ interface)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OpenJij/OpenJijTutorial/blob/master/source/ja/A2-CppIntroduction.ipynb)

この章ではOpenJijのcore interface (C++ interface)の紹介をします。
core python interfaceのみを取り扱いたい方はこの章をスキップしてしまっても問題ありません。

C++ interfaceはOpenJijの最下層レイヤーのAPIとなり、core python interfaceも内部でこのC++ interfaceを読み出しています。
pythonを使わずにC++のみを用いてOpenJijのパフォーマンスを最大限に引き出したい用途で使うことができます。


## 問題を投げてみる

まずはgithubからリポジトリをcloneしましょう

```sh
$ git clone https://github.com/OpenJij/OpenJij
$ cd OpenJij
```

OpenJijは基本的にヘッダオンリーなライブラリであるため、`src`ディレクトリのパスをコンパイル時に指定するだけでC++ interfaceが使えるようになります。
GPUアルゴリズム等を使う際にはライブラリをCMakeでビルドする必要がありますが、`build_gcc.sh`を実行すると自動でビルドを行うことができます。

前章のpython interfaceと全く同じ動作をするコードは次のように書くことができます。(`project_template/template.cpp`に同じ記述があります。)

```cpp
#include <graph/all.hpp>
#include <system/all.hpp>
#include <updater/all.hpp>
#include <algorithm/all.hpp>
#include <result/all.hpp>
#include <utility/schedule_list.hpp>
#include <random>

#include <iostream>

using namespace openjij;

int main(void){

    //まず相互作用行列を作成してあげます。Graphモジュールを使います。
    //問題サイズN=5の密結合グラフ(Dense)を定義します。
    constexpr std::size_t N = 5;
    auto dense = graph::Dense<double>(N);

    //相互作用を設定してあげます。
    for(std::size_t i=0; i<N; i++){
        for(std::size_t j=0; j<N; j++){
            dense.J(i, j) = (i == j) ? 0 : -1;
        }
    }

    //縦磁場を設定してあげます。
    for(std::size_t i=0; i<N; i++){
        dense.h(i) = -1;
    }

    //乱数生成器を定義します。(メルセンヌ・ツイスター)
    auto rand_engine = std::mt19937(0x1234);

    //続いてGraphから計算を行うためのSystemを作成します。
    //今回は通常の古典モンテカルロ計算のシステムを使用します。
    auto system = system::make_classical_ising(dense.gen_spin(rand_engine), dense);

    //アニーリングスケジュールを設定します。Utilityモジュールを使用します。
    auto schedule_list = utility::make_classical_schedule_list(0.1, 100, 10, 10);

    //実際にアニーリングを走らせます。Algorithmモジュールを使用します。
    //モンテカルロステップの更新方法として単純なSingleSpinFlipを用います。
    algorithm::Algorithm<updater::SingleSpinFlip>::run(system, rand_engine, schedule_list);

    //結果を取得します。
    std::cout << "The result spins are [";
    for(auto&& elem : result::get_solution(system)){
        std::cout << elem << " ";
    }

    std::cout << "]" << std::endl;
}
```

`project_template`内で`make`を実行し、`./tutorial`を実行すると同じように$[1,1,1,1,1]$ が解として出力されるのがわかります。
コンパイルオプションなどの}詳細については、`project_template`内の`Makefile`を参考にしてください。特にGPUを用いたアルゴリズムを利用する際には`build_gcc.sh`でビルドしたCUDAライブラリをリンクする必要がありますので注意してください。

前章のcore python interfaceのサンプルコードと見比べると分かるように、pythonインターフェースとほとんど同じように記述することが可能です。